Pseudocode according to RFC (https://datatracker.ietf.org/doc/html/rfc8439#section-2.5)

clamp(r): r &= 0x0ffffffc0ffffffc0ffffffc0fffffff
poly1305_mac(msg, key):
    r = le_bytes_to_num(key[0..15])
    clamp(r)
    s = le_bytes_to_num(key[16..31])
    a = 0  /* a is the accumulator */
    p = (1<<130)-5
    for i=1 upto ceil(msg length in bytes / 16)
        n = le_bytes_to_num(msg[((i-1)*16)..(i*16)] | [0x01])
        a += n
        a = (r * a) % p
        end
    a += s
    return num_to_16_le_bytes(a)
    end

In [65]:
from cryptography.hazmat.primitives.poly1305 import Poly1305

In [66]:
key_hex = """
fb e9 86 28 30 4d 29 a0 92 a6 9f a1 20 05 42 0b
0b 62 f2 c0 5d 88 d4 80 c5 d1 d2 10 62 23 8d 4d
"""

additional_data_hex = """
41 01 85 21 96 b6 46 22 e0 f5 ca 6b 44 95 af 62
20 07 19
"""

tag_hex = """
d6 24 e5 f8 cb b0 f4 02 8b 4c 8f ef 37 06 e5 b9
"""

cipher_text_hex = """
c0 19 77 2e db 35 3f e2 ef f4 49 9b ad 81 80 54
76 35 90 fb 97 29 dd e0 80 a0 86 22 7e 60 36 c8
23 37 52 b8 8c 1f 5d 93 27 1f 80 02 bc d2 f9 d8
01 df b3 51 49 e5 8b d5 24 0e 66 5a 3c 54 fc f5
6e 15 41 90 52 84 26 93 26 4e 19 6d 96 b6 90 bf
4f d9 d2 e2 b2 e8 a1 4c f2 f9 52 57 6a fe df 66
19 26 58 8a ae 88 c5 48 c0 92 83 4e 14 f1 b2 75
a0 a6 45 6d 90 a9 77 52 cf 7d ef ec f4 b9 a7 93
63 9f 2d 2a 20 87 b4 07 38 ee a8 82 aa 4e a9 59
26 a2 af 2a 46 35 41 96 f9 40 61 20 40 d2 f0 64
1c 18 71 3f 0d d9 4a 1b 8c 1c c8 49 09 e2 f4 8c
69 78 34 2d 10 7e ee c2 f1 d5 cf fc 53 81 8d ea
76 b2 70 ac c0 97 77 f9 e6 5c 12 f4 2c 67 e3 b6
3d 9a d3 d0 6e c1 00 54 a4 3d 7f 6c 83 5f 77 01
80 ad c8 3d 02 bb aa 2d eb 82 68 8d 3c cc dc 4a
bf 8e 9c 25 fe 74 59 44 8e 3a a0 d3 06 96 55 4e
75 ee 24 b1 3e fd f0 e2 d9 8c 54 32 9d d9 ce 21
0c 5a 62 27 9e a3 1b 3b 70 57 36 ea 49 45 24 83
5d 47 f7 5d d7 c6 85 a3 75 99 8e 7e 3e c0 dc 5a
d9 9f fc 87 55 51 95 bd 30 51 0e 65 3c f1 ea 23
1d f2 80 ca d3 b4 97 a7 54 00 5f 0a 9d e9 a5 fe
d0 f1 f6 60 72 d9 b2 fa 41 62 65 31 a9 cd 73 c7
ec a4 75 1d cf e8 c9 d2 dd ff 67 08 72 5b cc 9e
c5 2e d1 a2 79 d4 d4 0e 95 a5 89 5b 21 b2 ab 8e
08 7c 8a 25 3f e6 c5 99 be aa c8 66 e2 32 12 a6
e7 96 8e d9 ad cd c5 63 13 aa 3b f0 9c 3b dc b0
de 0a b6 33 03 f1 de 7b ab db 21 41 8c 4d 90 ad
0e 46 21 c7 03 73 7a fb 9a d7 81 25 f1 58 f5 74
b7 57 27 9b 53 e2 c4 5f 5b 6a 0b 3f 84 9f 6d 23
b8 d3 4c 12 6b 98 4c 37 78 1d d2 fd 38 4b f7 d9
e4 03 38 1e 64 b5 80 92 5d 2a 81 dc 62 34 3a d6
e3 a6 6d 2d 4b 69 bb 3a 06 72 7b 18 89 f8 e3 0e
67 c1 f2 9c a2 66 5c 45 86 84 51 3a 0e 42 71 e9
a6 da 86 58 e3 11 02 a0 68 7c 0a 86 7b 72 2d 5f
df 9e 37 4d e0 00 e5 a8 e9 eb 81 e2 47 b0 80 e4
cb a5 41 16 ad 10 ab 3c 8d e4 58 47 4a a1 88 7e
4f d1 a0 53 ba 90 6a 98 80 71 b5 c1 8a 14 ff 5d
9d d0 7c 4d 92 2a 80 0a 1d 57 a1 ba 1f 9c 1d 38
b2 f2 4e f4 77 42 a1 a8 79 63 56 cd e4 39 6a bc
a5 e8 ea a6 96 00 3a 15 ef b3 e3 58 2e 3f 6a 17
43 2c 63 43 6d 9c b9 92 0e 2f 1e 50 65 45 48 70
e5 aa 68 c7 f5 b6 76 10 b8 35 7e 3d 4c b8 d2 84
4f 19 3a 4c 49 56 4c 20 ff 36 b0 5f 09 cc ef 7e
56 1d f1 21 b7 11 3e 24 d1 f4 73 fd 0b d7 a2 1e
54 9a 45 c2 45 30 a3 5e ce 74 af ff 52 85 0e 4c
01 f5 a1 73 34 25 21 32 df d2 ec d6 1e 70 8d 69
41 25 83 7a e4 ea 7a 05 7b 2e a7 fc b4 af 8d a6
1f 10 52 27 af 41 96 e1 61 60 33 16 3e 57 39 81
bc 51 13 9d 1c 3b 24 db dd 41 84 41 9c fa 6d 12
22 0b e9 fd 2b db c7 bb 32 fc 67 bb a0 11 3c 4d
6c 1b d3 86 dd 19 e3 49 f0 17 bf d5 e5 0e 47 84
ed 23 5b 8a 04 2c 00 45 3b 66 3e 7a c4 ff 7e 90
88 8d 08 a0 62 e6 8f bb ed fb 6f 75 bc ba 5e 71
f5 3c 0e b7 35 7b 31 64 14 07 7f 92 02 13 d4 d9
ed 70 57 17 c5 4d 5b 63 ca 3a 74 22 5f b4 03 6c
cd 32 ee 8e 68 a4 e5 59 1a d5 45 a3 3e 20 34 d6
f7 4b cc f2 1c 4b 80 50 98 05 9f 80 64 0d e3 48
85 18 16 42 95 79 9a 32 51 3e a6 9e 45 6f d8 ca
c6 a5 41 e7 cc 16 bb e8 51 ca d9 41 2b af 2c e5
05 20 23 06 0e a0 eb 2e c0 6a 5f bc 34 03 13 b1
68 70 c1 2c 89 1e 1c 2b 20 9e ce 9f bd 61 b9 16
5c 6c 89 28 ba be ef 9b 04 81 6f c6 72 6e 50 a6
9d 9b 08 33 c2 cd 74 a6 bc 4e b4 3d c1 2f 84 16
15 15 70 a2 2c 9f 30 dd 96 80 d4 3c 2b 55 6f 1c
84 c0 7e 66 d9 76 d0 c8 a0 1d ac 9c 99 35 30 66
34 78 8a 78 59 48 da 1f 57 d8 bc 66 83 f3 a1 76
68 20 90 ef 71 6b c4 ea 3b 71 fb d3 d8 f8 24 ef
08 ea 52 06 46 b2 ff 56 d6 3c 9e 81 0b 5a 86 0c
be c4 db 23 3e 58 de d3 ae ba bd 47 8d c6 c9 37
3d 7b 89 80 53 fd 72 21 01 26 d6 80 60 6d c9 c5
2e 4d 81 69 4e 9f e6 67 22 e2 20 98 cd 0f 83 2f
8d 8c a6 cd 43 cf 3b 71 0d be d7 d7 0d 43 d0 7e
ce 4f cf a9 ed 61 71 51 43 d2 02 77 20 3b 0d 00
9c ae c1 6e de f0 56 f4 5b 1e a6 1d 00 aa 19 57
50 f3 69 a0 91 d6 60 02 71 05 2f 8a a0 7c e9 2d
a4 06 34 bc 6d 8a 89 2c 7d f4 83 88 22 a2 fa a9
ba
"""


In [67]:
key = bytes.fromhex(key_hex)
additional_data = bytes.fromhex(additional_data_hex)
cipher_text = bytes.fromhex(cipher_text_hex)
tag = bytes.fromhex(tag_hex)

total_data = b""

additional_data_padding = b"\x00" * (16 - len(additional_data) % 16) if len(additional_data) % 16 != 0 else b""
cipher_text_padding = b"\x00" * (16 - len(cipher_text) % 16) if len(cipher_text) % 16 != 0 else b""

little_endian_additional_data_len = len(additional_data).to_bytes(8, "little")
little_endian_cipher_text_len = len(cipher_text).to_bytes(8, "little")

total_data += additional_data + additional_data_padding
total_data += cipher_text + cipher_text_padding
total_data += little_endian_additional_data_len
total_data += little_endian_cipher_text_len

# Creating the Poly1305 MAC
poly1305 = Poly1305.generate_tag(key, total_data)

# Verifying the Poly1305 MAC
try:
    Poly1305.verify_tag(key, total_data, tag)
    print("Verification successful!")
except Exception as e:
    print(f"Verification failed: {e}")


Verification successful!


In [68]:
tags = []

tags.append(Poly1305.generate_tag(key, total_data))
tags.append(tag)

for i, t in enumerate(tags):
    print(f"Tag {i}: {t.hex()}")

Tag 0: d624e5f8cbb0f4028b4c8fef3706e5b9
Tag 1: d624e5f8cbb0f4028b4c8fef3706e5b9


In [69]:
def clamp(r):
    r &= 0x0ffffffc0ffffffc0ffffffc0fffffff
    return r

def my_poly1305(key, data):
    r = int.from_bytes(key[:16], "little")
    r = clamp(r)
    s = int.from_bytes(key[16:], "little")
    a = 0
    p = (1 << 130) - 5
    for i in range(1, len(data) // 16 + 1):
        n = int.from_bytes(data[(i - 1) * 16:i * 16] + b"\x01", "little")
        a += n
        a = (r * a) % p
    a += s
    tag_32 = a.to_bytes(20, "little")
    return tag_32[:16]

my_tag = my_poly1305(key, total_data)
print(f"My tag: {my_tag.hex()}")

My tag: d624e5f8cbb0f4028b4c8fef3706e5b9


In [70]:
import random

In [82]:
a_128_bit = random.getrandbits(128)
b_128_bit = random.getrandbits(128)


print(f"a_128_bit: \t\t{a_128_bit}")
print(f"b_128_bit: \t\t{b_128_bit}")
print(f"a * b as 128: \t\t{(a_128_bit * b_128_bit) & (1 << 129 - 1)}")

a_128_bit: 		170615571459067097002981904407189558596
b_128_bit: 		110018957958747675243191188450643356575
a * b as 128: 		340282366920938463463374607431768211456


In [83]:
a_lower_64 = a_128_bit & ((1 << 64) - 1)
a_higher_64 = a_128_bit >> 64
b_lower_64 = b_128_bit & ((1 << 64) - 1)
b_higher_64 = b_128_bit >> 64

print(f"a_lower_64: \t\t{a_lower_64}")
print(f"a_higher_64: \t\t{a_higher_64}")
print(f"together: \t\t{a_higher_64 << 64 | a_lower_64}")
assert a_128_bit == (a_higher_64 << 64 | a_lower_64)
print(f"b_lower_64: \t\t{b_lower_64}")
print(f"b_higher_64: \t\t{b_higher_64}")
print(f"together: \t\t{b_higher_64 << 64 | b_lower_64}")
assert b_128_bit == (b_higher_64 << 64 | b_lower_64)

a_lower_64: 		17026492550280942916
a_higher_64: 		9249088661788818605
together: 		170615571459067097002981904407189558596
b_lower_64: 		11501590615897746335
b_higher_64: 		5964139661673280390
together: 		110018957958747675243191188450643356575


In [87]:
def mul_128_based_on_64(a_lo, a_hi, b_lo, b_hi):
    a = a_lo * b_lo
    a = a + ((a_hi * b_lo) << 64)
    a = a + ((a_lo * b_hi) << 64)
    # a = a + ((a_hi * b_hi) << 128)
    return a

In [88]:
res = mul_128_based_on_64(a_lower_64, a_higher_64, b_lower_64, b_higher_64) & (1 << 129 - 1)

print(res)
assert res == (a_128_bit * b_128_bit) & (1 << 129 - 1) 

340282366920938463463374607431768211456


In [90]:
# modulo (2^130 - 5) with easiy bitwise operations

def mod_p(x):
    p = (1 << 130) - 5
    while x >= p:
        x = (x & p) + (x >> 130)
    return x

In [91]:
random_150_bit = random.getrandbits(150)
print(f"random_150_bit: {random_150_bit}")
p = (1 << 130) - 5
assert mod_p(random_150_bit) == random_150_bit % p

random_150_bit: 56331756153269065504165927485348244966261819


AssertionError: 

In [103]:
a = (0x0214_4560_89a0_0000 % 0xffff)
# a as hex
print(f"a: {a:x}")

a: d114


In [109]:
p = (1 << 130) - 5
lut = {}
for i in range(256):
    power_i = 1 << i
    lut[power_i] = power_i % p

In [112]:
# # iterate over lut
# for k, v in lut.items():
#     print(f"{k:02x}: {v:02x}")

In [113]:
random_256_bit_number = random.getrandbits(256)

In [114]:
random_256_bit_number % p

1086494840599600104323888853743677681315

In [116]:
tmp = 0
for i in range(256):
    if (random_256_bit_number >> i) & 1:
        tmp = tmp + lut[1 << i]

tmp

1086494840599600104323888853743677681315

In [127]:
print("{")
i = 0
for k, v in lut.items():
    hi = v >> 128
    mid = (v >> 64) & ((1 << 64) - 1)
    lo = v & ((1 << 64) - 1)
    print(f"0x{hi:016x}, 0x{mid:016x}, 0x{lo:016x}, // for k = {k:d}")
print("};")

{
0x0000000000000000, 0x0000000000000000, 0x0000000000000001, // for k = 1
0x0000000000000000, 0x0000000000000000, 0x0000000000000002, // for k = 2
0x0000000000000000, 0x0000000000000000, 0x0000000000000004, // for k = 4
0x0000000000000000, 0x0000000000000000, 0x0000000000000008, // for k = 8
0x0000000000000000, 0x0000000000000000, 0x0000000000000010, // for k = 16
0x0000000000000000, 0x0000000000000000, 0x0000000000000020, // for k = 32
0x0000000000000000, 0x0000000000000000, 0x0000000000000040, // for k = 64
0x0000000000000000, 0x0000000000000000, 0x0000000000000080, // for k = 128
0x0000000000000000, 0x0000000000000000, 0x0000000000000100, // for k = 256
0x0000000000000000, 0x0000000000000000, 0x0000000000000200, // for k = 512
0x0000000000000000, 0x0000000000000000, 0x0000000000000400, // for k = 1024
0x0000000000000000, 0x0000000000000000, 0x0000000000000800, // for k = 2048
0x0000000000000000, 0x0000000000000000, 0x0000000000001000, // for k = 4096
0x0000000000000000, 0x000000000

In [ ]:
# https://electronics.stackexchange.com/questions/608840/verilog-modulus-operator-for-non-power-of-two-synthetizable/608854#608854